In [1]:
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error,max_error

def split_x_and_y(database,x,y):
  dataset_x = database[x]
  dataset_y = database[y]
  return dataset_x,dataset_y


def scores(Y_true, Y_predicted):
  r2 = r2_score(Y_true, Y_predicted)
  meansquarederror = mean_squared_error(Y_true, Y_predicted)
  meanabsoluteerror = mean_absolute_error(Y_true, Y_predicted)
  maxerror = max_error(Y_true, Y_predicted)

  print('r2:',r2,'meansquarederror:',meansquarederror,'meanabsoluteerror:',meanabsoluteerror,'maxerror:',maxerror)
  return r2,meansquarederror,meanabsoluteerror,maxerror

# NUSSELT

In [37]:
import pandas as pd
database_pre = pd.read_excel('database_TCC.xlsx')
database = database_pre[['A','E','N','Nu']]
database

,A,E,N,Nu
0,25,1,11,78.62
1,25,1,5,102.00
2,25,2,8,91.57
3,25,3,11,31.00
4,25,3,5,88.97
5,35,1,8,96.90
6,35,2,11,61.80
7,35,2,5,93.30
8,35,3,8,76.90
9,25,1,8,98.34


In [38]:
import pickle

with open("standard_scaler_Nu.pkl", "rb") as f:
    rawdata = f.read()

standard_scaler_nu = pickle.loads(rawdata)
data = standard_scaler_nu.transform(database)
database = pd.DataFrame(data,columns=database.columns)

### Decision Tree Regressor

In [4]:
with open("Decision_Tree_Regressor_Nu.pkl", "rb") as f:
    rawdata = f.read()
decision_tree_regressor_nu = pickle.loads(rawdata)

In [5]:
dataset_x,dataset_y = split_x_and_y(database.iloc[[0,1,2,3,4,5,6,7,8]],['A','E','N'],'Nu')
test_x,test_y = split_x_and_y(database.iloc[[9,10,11]],['A','E','N'],'Nu')

decision_tree_regressor_nu.fit(dataset_x,dataset_y)
decision_tree_regressor_nu.score(test_x,test_y)

0.9897242898141063

In [6]:
desnormalizar = test_x.copy()
desnormalizar['Nu'] = decision_tree_regressor_nu.predict(test_x)
desnormalizado_teste = standard_scaler_nu.inverse_transform(desnormalizar)
desnormalizado_teste

array([[ 25. ,   1. ,   8. , 102. ],
       [ 25. ,   2. ,   5. ,  93.3],
       [ 35. ,   3. ,  11. ,  31. ]])

In [7]:
decision_tree_results = desnormalizado_teste[:,-1]

In [8]:
desnormalizar = test_x.copy()
desnormalizar['Nu'] = test_y
desnormalizado_resultado = standard_scaler_nu.inverse_transform(desnormalizar)
desnormalizado_resultado

array([[25.  ,  1.  ,  8.  , 98.34],
       [25.  ,  2.  ,  5.  , 89.62],
       [35.  ,  3.  , 11.  , 31.47]])

### Linear Regression with Polynomial Preprocessing

In [9]:
with open("Linear_Regression_with_Poly_Nu.pkl", "rb") as f:
    rawdata = f.read()
linear_regression_nu = pickle.loads(rawdata)

In [10]:
dataset_x,dataset_y = split_x_and_y(database.iloc[[0,1,2,3,4,5,6,7,8]],['A','E','N'],'Nu')
test_x,test_y = split_x_and_y(database.iloc[[9,10,11]],['A','E','N'],'Nu')

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

poly = PolynomialFeatures(4,interaction_only=False,include_bias=False)
LR = LinearRegression()

dataset_x_transformed = poly.fit_transform(dataset_x)
test_x_transformed = poly.fit_transform(test_x)

linear_regression_nu.score(test_x_transformed,test_y)

0.96034797005603

In [11]:
desnormalizar = test_x.copy()
desnormalizar['Nu'] = linear_regression_nu.predict(test_x_transformed)
desnormalizado_teste = standard_scaler_nu.inverse_transform(desnormalizar)
desnormalizado_teste

array([[25.        ,  1.        ,  8.        , 96.64663975],
       [25.        ,  2.        ,  5.        , 94.93308063],
       [35.        ,  3.        , 11.        , 40.05514906]])

In [12]:
linear_regression_results = desnormalizado_teste[:,-1]

In [13]:
desnormalizar = test_x.copy()
desnormalizar['Nu'] = test_y
desnormalizado_resultado = standard_scaler_nu.inverse_transform(desnormalizar)
desnormalizado_resultado

array([[25.  ,  1.  ,  8.  , 98.34],
       [25.  ,  2.  ,  5.  , 89.62],
       [35.  ,  3.  , 11.  , 31.47]])

# Keras

In [27]:
from keras.models import load_model
keras_nu = load_model('keras_model_Nu.h5')

dataset_x,dataset_y = split_x_and_y(database.iloc[[0,1,2,3,4,5,6,7,8]],['A','E','N'],'Nu')
test_x,test_y = split_x_and_y(database.iloc[[9,10,11]],['A','E','N'],'Nu')

In [28]:
desnormalizar = test_x.copy()
desnormalizar['Nu'] = keras_nu.predict(test_x.values)
desnormalizado_teste = standard_scaler_nu.inverse_transform(desnormalizar)
desnormalizado_teste

1/1 [==============================] - 0s 51ms/step


array([[25.        ,  1.        ,  8.        , 96.37617291],
       [25.        ,  2.        ,  5.        , 96.63502646],
       [35.        ,  3.        , 11.        , 32.7789412 ]])

In [29]:
keras_results = desnormalizado_teste[:,-1]

In [31]:
desnormalizar = test_x.copy()
desnormalizar['Nu'] = test_y
desnormalizado_resultado = standard_scaler_nu.inverse_transform(desnormalizar)
desnormalizado_resultado

answers = desnormalizado_resultado[:,-1]

In [32]:
scores(desnormalizado_resultado[:,-1],desnormalizado_teste[:,-1])

r2: 0.9792734994843473 meansquarederror: 18.260180064482245 meanabsoluteerror: 3.4292649169595086 maxerror: 7.015026464920979


(0.9792734994843473, 18.260180064482245, 3.4292649169595086, 7.015026464920979)

# RESULTS

In [43]:
df_results_nu = pd.DataFrame(database_pre.iloc[[9,10,11]][['A','E','N','Nu']])
df_results_nu['Decision Tree Regressor'] = decision_tree_results
df_results_nu['Linear Regression'] = linear_regression_results
df_results_nu['Keras'] = keras_results

In [44]:
df_results_nu

,A,E,N,Nu,Decision Tree Regressor,Linear Regression,Keras
9,25,1,8,98.34,102.0,96.646640,96.376173
10,25,2,5,89.62,93.3,94.933081,96.635026
11,35,3,11,31.47,31.0,40.055149,32.778941


In [46]:
df_results_nu['average'] = df_results_nu[['Decision Tree Regressor','Linear Regression','Keras']].mean(axis=1)

In [47]:
df_results_nu

,A,E,N,Nu,Decision Tree Regressor,Linear Regression,Keras,average
9,25,1,8,98.34,102.0,96.646640,96.376173,98.340938
10,25,2,5,89.62,93.3,94.933081,96.635026,94.956036
11,35,3,11,31.47,31.0,40.055149,32.778941,34.611363


In [49]:
scores(df_results_nu['Nu'],df_results_nu['average'])

r2: 0.9854933172572207 meansquarederror: 12.780480661528875 meanabsoluteerror: 2.8261122236721383 maxerror: 5.336035698058936


(0.9854933172572207, 12.780480661528875, 2.8261122236721383, 5.336035698058936)